### Structured Output with Groq and Instructor

While Large Language Models (LLMs) are often employed for building chatbots or conversational agents, numerous real-world applications require a different approach - one that goes beyond mere dialogue and involves producing structured, machine-readable outputs.

Consider a typical scenario: we want to produce structured JSON data from an LLM. While tools like Python's json module allow us to handle this data, they also come with their own set of challenges, such as validating data types and ensuring consistency across outputs. Manually checking these aspects can be tedious and error-prone. LLMs also tend to forget to include a comma or a closing bracket ('}') somewhere in the produced JSON from time to time, which would invalidate the whole JSON output.

1. A Very Simple Use Case
Let's dive right into how you can set up the instructor library with models powered by Groq to generate structured JSON outputs. We'll keep it simple and straightforward so you can get up and running quickly.

### Lets Dive into Code

#### Installing the Necessary Libraries
Install the required Python libraries. You'll need:
<ul>
<li>groq </li>
<li>instructor </li>
<li>python-dotenv (for loading environment variables) </li>
</ul>

In [1]:
pip install -U  instructor 

  Attempting uninstall: tenacity
    Found existing installation: tenacity 8.5.0
    Uninstalling tenacity-8.5.0:
      Successfully uninstalled tenacity-8.5.0
  Attempting uninstall: jiter
    Found existing installation: jiter 0.5.0
    Uninstalling jiter-0.5.0:
      Successfully uninstalled jiter-0.5.0
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-core 0.11.16 requires tenacity!=8.4.0,<9.0.0,>=8.2.0, but you have tenacity 9.0.0 which is incompatible.
llama-index-readers-file 0.2.2 requires pypdf<5.0.0,>=4.0.1, but you have pypdf 5.1.0 which is incompatible.
llama-index-legacy 0.9.48.post3 requires tenacity<9.0.0,>=8.2.0, but you have tenacity 9.0.0 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 24.3.1

In [2]:
import instructor
from dotenv import load_dotenv
from pydantic import BaseModel
from groq import Groq

# Load the Groq API key from .env file
load_dotenv()

# Describe the desired output schema using pydantic models
class UserInfo(BaseModel):
    name: str
    age: int
    email: str

# The text to extract data from
text = """
John Doe, a 35-year-old software engineer from New York, has been working with large language models for several years.
His email address is johndoe@example.com.
"""

# Patch Groq() with instructor, this is where the magic happens!
client = instructor.from_groq(Groq(), mode=instructor.Mode.JSON)

# Call the API
user_info = client.chat.completions.create(
    model="llama-3.1-70b-versatile",
    response_model=UserInfo, # Specify the response model
    messages=[
        {"role": "system", "content": "Your job is to extract user information from the given text."},
        {"role": "user", "content": text}
    ],
    temperature=0.65,
)

print(f"Name: {user_info.name}")
print(f"Age: {user_info.age}")
print(f"Email: {user_info.email}")

Name: John Doe
Age: 35
Email: johndoe@example.com
